In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [2]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [4]:
columns = {
    'input_ids': 'uint16',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [6]:
split_by = 1000000

In [7]:
!mkdir partitions-starcoder

mkdir: cannot create directory ‘partitions-starcoder’: File exists


In [8]:
index = 0
count = 0
a = open(f'partitions-starcoder/combined-lm-{index}.jsonl', 'w')

with open('prepare-starcoder.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-starcoder/combined-lm-{index}.jsonl', 'w')
            
a.close()

34721482it [22:54, 25253.13it/s]


In [9]:
block_size = 4096

def read_dataset(train_file, block_size = block_size):
    
    tokenizer = AutoTokenizer.from_pretrained(
        'malaysia-ai/bpe-tokenizer',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    text_column_name = 'text'
    temp = []
    with open(train_file) as fopen:
        for l in fopen:
            l = msgspec.json.decode(l)
            tokenized = tokenizer(l['text'])['input_ids']
            temp.extend(tokenized)
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    yield np.array(block).astype(np.uint16)

In [10]:
files = sorted(glob('partitions-starcoder/combined-lm-*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
files

['partitions-starcoder/combined-lm-0.jsonl',
 'partitions-starcoder/combined-lm-1.jsonl',
 'partitions-starcoder/combined-lm-2.jsonl',
 'partitions-starcoder/combined-lm-3.jsonl',
 'partitions-starcoder/combined-lm-4.jsonl',
 'partitions-starcoder/combined-lm-5.jsonl',
 'partitions-starcoder/combined-lm-6.jsonl',
 'partitions-starcoder/combined-lm-7.jsonl',
 'partitions-starcoder/combined-lm-8.jsonl',
 'partitions-starcoder/combined-lm-9.jsonl',
 'partitions-starcoder/combined-lm-10.jsonl',
 'partitions-starcoder/combined-lm-11.jsonl',
 'partitions-starcoder/combined-lm-12.jsonl',
 'partitions-starcoder/combined-lm-13.jsonl',
 'partitions-starcoder/combined-lm-14.jsonl',
 'partitions-starcoder/combined-lm-15.jsonl',
 'partitions-starcoder/combined-lm-16.jsonl',
 'partitions-starcoder/combined-lm-17.jsonl',
 'partitions-starcoder/combined-lm-18.jsonl',
 'partitions-starcoder/combined-lm-19.jsonl',
 'partitions-starcoder/combined-lm-20.jsonl',
 'partitions-starcoder/combined-lm-21.jsonl'

In [11]:
next(read_dataset(files[1]))

array([   1, 5565, 9666, ..., 3626,   18,  862], dtype=uint16)

In [13]:
!rm -rf tokenized_starcoder
!mkdir tokenized_starcoder

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
def loop(files):
    files, index = files
    out_root = f'tokenized_starcoder/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=compression, hashes=hashes) as out:
        for f in files:
            for block in tqdm(read_dataset(train_file = f)):
                sample = {
                    'input_ids': block
                }
                out.write(sample)

In [15]:
import mp
mp.multiprocessing(files, loop, cores = 20, returned = False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [17]:
from streaming import StreamingDataset

total = 0
for f in glob('tokenized_starcoder/tokenized-*'):
    dataset = StreamingDataset(local = f)
    total += len(dataset)
total * block_size

40981254144